## 仅供交叉验证 前馈神经网络（NNAR）

In [2]:
import pickle
import numpy as np
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [2]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [3]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [4]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 构建模型

In [3]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

def build_direct_dnn_model():
    model = keras.models.Sequential()
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(5))
    
    optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    for train, test in kfold.split(X, y_cat):
        model = build_direct_dnn_model()
        history = model.fit(X[train], y[train], epochs=100, batch_size=16, verbose=1, validation_data=(X[test], y[test]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])

        y_test = y[test]
        y_pred = model.predict(X[test]).reshape(y[test].shape)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('mlp_metrics.dict', 'wb'))
    
    return metrics

In [6]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 2112 samples, validate on 531 samples
Epoch 1/100
2112/2112 [==============================] - 1s 595us/sample - loss: 0.9757 - val_loss: 0.8962
Epoch 2/100
2112/2112 [==============================] - 0s 161us/sample - loss: 0.9062 - val_loss: 0.8870
Epoch 3/100
2112/2112 [==============================] - 0s 165us/sample - loss: 0.8989 - val_loss: 0.8839
Epoch 4/100
2112/2112 [==============================] - 0s 162us/sample - loss: 0.8955 - val_loss: 0.8843
Epoch 5/100
2112/2112 [==============================] - 0s 163us/sample - loss: 0.8923 - val_loss: 0.8842
Epoch 6/100
2112/2112 [==============================] - 0s 162us/sample - loss: 0.8901 - val_loss: 0.8838
Epoch 7/100
2112/2112 [==============================] - 0s 161us/sample - loss: 0.8883 - val_loss: 0.8837
Epoch 8/100
2112/2112 [==============================] - 0s 163us/sample - loss: 0.8862 - val_loss: 0.8845
Epo

2115/2115 [==============================] - 0s 207us/sample - loss: 0.8748 - val_loss: 0.8958
Epoch 13/100
2115/2115 [==============================] - 0s 207us/sample - loss: 0.8741 - val_loss: 0.8969
Epoch 14/100
2115/2115 [==============================] - 0s 203us/sample - loss: 0.8718 - val_loss: 0.8969
Epoch 15/100
2115/2115 [==============================] - 0s 204us/sample - loss: 0.8698 - val_loss: 0.8980
Epoch 16/100
2115/2115 [==============================] - 0s 202us/sample - loss: 0.8689 - val_loss: 0.8984
Epoch 17/100
2115/2115 [==============================] - 0s 203us/sample - loss: 0.8668 - val_loss: 0.8955
Epoch 18/100
2115/2115 [==============================] - 0s 206us/sample - loss: 0.8647 - val_loss: 0.9000
Train on 2116 samples, validate on 527 samples
Epoch 1/100
2116/2116 [==============================] - 1s 509us/sample - loss: 0.9410 - val_loss: 0.9212
Epoch 2/100
2116/2116 [==============================] - 0s 204us/sample - loss: 0.8951 - val_loss: 0.9

Epoch 15/100
4111/4111 [==============================] - 1s 199us/sample - loss: 0.5509 - val_loss: 0.6115
Epoch 16/100
4111/4111 [==============================] - 1s 197us/sample - loss: 0.5487 - val_loss: 0.6098
Epoch 17/100
4111/4111 [==============================] - 1s 195us/sample - loss: 0.5458 - val_loss: 0.6077
Epoch 18/100
4111/4111 [==============================] - 1s 196us/sample - loss: 0.5432 - val_loss: 0.6076
Epoch 19/100
4111/4111 [==============================] - 1s 195us/sample - loss: 0.5406 - val_loss: 0.6086
Epoch 20/100
4111/4111 [==============================] - 1s 195us/sample - loss: 0.5365 - val_loss: 0.6127
Epoch 21/100
4111/4111 [==============================] - 1s 197us/sample - loss: 0.5356 - val_loss: 0.6120
Epoch 22/100
4111/4111 [==============================] - 1s 196us/sample - loss: 0.5325 - val_loss: 0.6113
Epoch 23/100
4111/4111 [==============================] - 1s 197us/sample - loss: 0.5298 - val_loss: 0.6118
Train on 4112 samples, valid

4115/4115 [==============================] - 1s 204us/sample - loss: 0.5631 - val_loss: 0.6143
Epoch 11/100
4115/4115 [==============================] - 1s 216us/sample - loss: 0.5603 - val_loss: 0.6159
Epoch 12/100
4115/4115 [==============================] - 1s 215us/sample - loss: 0.5575 - val_loss: 0.6112
Epoch 13/100
4115/4115 [==============================] - 1s 202us/sample - loss: 0.5552 - val_loss: 0.6116
Epoch 14/100
4115/4115 [==============================] - 1s 225us/sample - loss: 0.5513 - val_loss: 0.6141
Epoch 15/100
4115/4115 [==============================] - 1s 205us/sample - loss: 0.5490 - val_loss: 0.6131
Epoch 16/100
4115/4115 [==============================] - 1s 210us/sample - loss: 0.5454 - val_loss: 0.6151
Epoch 17/100
4115/4115 [==============================] - 1s 223us/sample - loss: 0.5436 - val_loss: 0.6172
Epoch 18/100
4115/4115 [==============================] - 1s 214us/sample - loss: 0.5410 - val_loss: 0.6118
Epoch 19/100
4115/4115 [=================

In [6]:
metrics

{'gene': {'overall': {'mae': 0.7299490896746337,
   'rmse': 1.2918968704150449,
   'ndcg': 0.44835543717483367},
  'annual': {'mae': array([0.41098335, 0.57326106, 0.75468637, 0.88329084, 1.02752384]),
   'rmse': array([0.74612466, 0.97067574, 1.33166402, 1.48635163, 1.66849718]),
   'ndcg': array([0.46505046, 0.36096419, 0.14908492, 0.19261638, 0.15743147])}},
 'transplant': {'overall': {'mae': 0.7718043551135649,
   'rmse': 1.2752554128409044,
   'ndcg': 0.48122490028670706},
  'annual': {'mae': array([0.75218725, 0.77410503, 0.74279975, 0.7759474 , 0.81398235]),
   'rmse': array([1.30078556, 1.29846621, 1.21739863, 1.22831554, 1.31962679]),
   'ndcg': array([0.09773368, 0.07247887, 0.02053076, 0.10604713, 0.11611737])}}}

In [7]:
metrics

{'gene': {'overall': {'mae': 0.7299490896746337,
   'rmse': 1.2918968704150449,
   'ndcg': 0.44835543717483367,
   'mape': 4.43813367441968,
   'r2': 0.2366877334464476,
   'pearson': 0.5213191822517468,
   'acc': 0.34326412607594264},
  'annual': {'mae': array([0.41098335, 0.57326106, 0.75468637, 0.88329084, 1.02752384]),
   'rmse': array([0.74612466, 0.97067574, 1.33166402, 1.48635163, 1.66849718]),
   'ndcg': array([0.46505046, 0.36096419, 0.14908492, 0.19261638, 0.15743147]),
   'mape': array([3.48005609, 2.8328771 , 3.14869389, 6.23453968, 6.49450161]),
   'r2': array([0.43897391, 0.27529658, 0.16778358, 0.05561811, 0.01792775]),
   'pearson': array([0.67472678, 0.56063183, 0.46457533, 0.32847791, 0.26389998]),
   'acc': array([0.60726591, 0.22830786, 0.23867794, 0.33481486, 0.30725406])}},
 'transplant': {'overall': {'mae': 0.7718043551135649,
   'rmse': 1.2752554128409044,
   'ndcg': 0.48122490028670706,
   'mape': 3.659302529390196,
   'r2': 0.4193337345315111,
   'pearson': 0.